In [1]:
%run StdPackages.ipynb
d['rawData'] = os.path.join(d['data'],'rawData69') # add
d['processedData'] = os.path.join(d['data'],'processedData') # update to raw data folder
os.chdir(d['py'])
from loadIO import *
import RAS
from scipy.cluster.vq import kmeans2

No clean-up of work-folder


# Aggregate IO data for WasteCGE

We base the aggregation on two things: First, we outline the specific sectors that we draw upon in the model. This includes energy (sector code 35000) and waste collection, treatment, disposal (sector code 37390). For every other sector, we define input shares and use KMeans to aggregate sectors into 2 residual sectors. This uses the same approach as in ```AggIO.ipynb``` and then ```modelDataFromIO.ipynb``` (which contains a bit more explanation along the way).

## 1. Aggregate IO data

*Load full data:*

In [2]:
fullIO = os.path.join(d['processedData'], 'IO69')
t0 = 2019
sectorsKMeans = 3 # beyond this we have energy and waste sectors
name = 'IO_WCGE' # add new name for the database when exporting
db = GpyDB(fullIO, name = name)

*Mappings:*

In [3]:
mDur = pd.MultiIndex.from_tuples([(k, 'K') for k in db('s_i')], names = ['n','nn'])
m = pd.MultiIndex.from_tuples([('35000', 'Energy'), ('37390', 'Waste')], names = ['s','ss'])

*Create input intensities to aggregate sectors using KMeans*

In [4]:
vS = adj.rc_pd(pdSum(db('vD'), 's'), db('n_p')).rename_axis(index = {'n':'s'})
vD = adj.rc_pd(db('vD'), db('s_p'))
inpIntensity = vD / vS
inpIntensity = inpIntensity / pdSum(inpIntensity, 'n')
μ = adj.rc_pd(inpIntensity, ('not', m)).xs(t0).unstack('n').fillna(0)
km = kmeans2(μ, sectorsKMeans)
mKM = pd.MultiIndex.from_arrays([μ.index, 'agg'+pd.Index(km[1], name = 'ss').astype(str)])
m = m.union(mKM)

*Create I_K (flow investment variable corresponding to durable stock $K$)*:

In [5]:
mDur_s = mDur.set_levels(mDur.levels[-1].map(lambda x: 'I_'+x), level = 1).rename(['s','ss'])

*Full sector mapping:*

In [6]:
ms = m.union(mDur_s)
ms_neutral = pd.MultiIndex.from_arrays([db('s'), db('s').rename('ss')]) # neutral mapping from (x,x)
m_sector = ms.union(adj.rc_pd(ms_neutral, ('not', ms.levels[0]))) # full mapping: Use specific mapping if this exists, otherwise default to neutral mapping.

*Apply mapping:*

In [7]:
AggDB.aggDB(db, m_sector);

*Adjust goods index:*

In [8]:
mn = m.union(m.set_levels([l+'_F' for l in m.levels])).rename(['n','nn']).union(mDur)
mn_neutral = pd.MultiIndex.from_arrays([db('n'), db('n').rename('nn')])
m_goods = mn.union(adj.rc_pd(mn_neutral, ('not', mn.levels[0])))

*Apply mapping:*

In [9]:
AggDB.aggDB(db, m_goods);

## 2. Model data

*Clean up some data:*

In [10]:
[db.series.database.pop(k) for k in ('gc','vC','vC_tax')];
[db.__setitem__(k, db(k)[db(k)!=0]) for k in db.getTypes(['var'])];

IO data is measured in mio DKK. Here, we rescale to billion DKK instead (*Note: This rescales all variables in the database, so it is assumed that everything is measured in absolute values (and not e.g. ratios)*)

In [11]:
[db.__setitem__(k, db(k)/1000) for k in db.getTypes(['var'])];

Translate depreciation of durables to rates, distinguish between investments and durables (flow, stock) with investment good syntax ```I_x``` for durable ```x```. Define mapping ```dur2inv``` and subsets ```dur_p, inv_p```. Add investments and value of durables to the vector ```vD```:

In [12]:
db['rDepr'] = db('vD_depr')/db('vD_dur')
db['dur_p'] = db('vD_dur').index.levels[db['vD_dur'].domains.index('n')]
db['inv_p'] = db('dur_p').map(lambda x: f'I_{x}')
db['dur2inv'] = pd.MultiIndex.from_arrays([db('dur_p'), db('inv_p').rename('nn')])
db('vD_inv').index = db('vD_inv').index.set_levels(db('vD_inv').index.levels[db['vD_inv'].domains.index('n')].map(lambda x: f'I_{x}'), level = 'n')
db['vD'] = db('vD_inv').combine_first(db('vD')).combine_first(db('vD_dur'))

IO data is measured in mio DKK. Here, we rescale to billion DKK instead:

### RAS

Simple RAS algorithm:

In [13]:
threshold = 1 # remove values less than billion
v0 = adj.rc_pd(db('vD'), ('and', [('or', [db('n_p'), db('n_F')]),
                                  ('or', [db('s_p'), db('s_i')])]))
leaveCols = db('n_F') # are there any type of goods that we do not need to balance
leaveRows = None # are there any type of sectors that we do not need to balance
vBar = v0[v0<threshold] * 0

*Get RAS adjustments:*

In [14]:
vD = RAS.simpleRAS(v0, vBar, leaveCols = leaveCols, leaveRows = leaveRows, tol = 1e-8, iterMax = 1000)

Largest deviation summing over n: 7.99172994447872e-09
Largest deviation summing over s: 2.2737367544323206e-13


*Merge things back up again:*

In [15]:
vD_full = vD.combine_first(db('vD'))
vD_full = vD_full[vD_full!=0] # remove zero values again

*Remove residual income category (we don't currently use this in the model, this will enter the return on durables instead):*

In [16]:
db['vD'] = adj.rc_pd(vD_full, ('not', pd.Index(['resIncome'], name = 'n')))

### Create other variables

In [17]:
db['R_LR'] = gpy(1.03, name = 'R_LR', type = 'par')
db['infl_LR'] = gpy(0.02, name = 'infl_LR', type = 'par')
db['g_LR'] = gpy(0.02, name = 'g_LR', type = 'par')
model_vS(db)
model_p(db)
model_durables(db, db('R_LR'), db('infl_LR'))
model_quantNonDurables(db) 

### Create other subsets and mappings

Subsets of goods/sectors:

In [18]:
db['nEqui'] = db('vS').index.droplevel('s').unique() # what goods require an equilibrium condition
db['d_qS'] = db['vS'].index 
db['d_qD'] = adj.rc_pd(db('vD'), db('nEqui')).index 
db['d_qSEqui'] = adj.rc_pd(db['d_qS'].vals, ('not', db('s_HH'))) # Subset of qS values to be endogenized in general equilibrium
db['d_pEqui'] = pd.Index(['L'], name ='n') # Subset of prices to be endogenized in general equilibrium 

####  Trade mappings

Define the mappings:
* ```dom2for[n,nn]```: Mapping from domestic to the equivalent foreign goods (with syntax ```x,x_F```).
* ```dExport[t,s,n]```: Foreign sectors' demand for domestic goods.
* ```dImport[t,s,n,nn]```: sector, domestic good, foreign good combinations in data - i.e. where a sector demands both domestic and foreign type of product.
* ```dImport_dom[t,s,n]```: sector, domestic good combination (s,n) where the sector only demands the domestic and not the corresponding foreign good.
* ```dImport_for[t,s,n]```: sector, foreign good combinations (s,n) where the sector only demand the foreign and not the corresponding domestic good.

In [19]:
db['dom2for'] = pd.MultiIndex.from_arrays([db('n_p').sort_values(), db('n_F').sort_values().rename('nn')])
db['dExport'] = adj.rc_pd(db('vD'), db('s_f')).index
vD_dom = stdSort(adjMultiIndex.applyMult(adj.rc_pd(db('vD'), db('n_p')), db('dom2for')))
vD_for = adj.rc_pd(db('vD'), db('n_F')).rename_axis(index= {'n':'nn'})
db['dImport'] = stdSort(adj.rc_pd(vD_dom, vD_for)).index
db['dImport_dom'] = adj.rc_pd(vD_dom, ('not', vD_for)).droplevel('nn').index
db['dImport_for'] = adj.rc_pd(vD_for, ('not', db('dImport'))).rename_axis(index = {'nn':'n'}).index

### Export

In [20]:
AggDB.updSetsFromSyms(db, types = ['var','par','map']) # define sets from variables/parameters defined throughout
db.export()